In [ ]:
import scipy.special as sp
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.optimize import minimize,least_squares,Bounds
from scipy.special import gamma
import matplotlib.cm as cm
from Boruvka_Mod import Graph
import utm
import scipy.interpolate as spline
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
### Sets model parameters
v0=8.0
nwind = 33
wind=np.arange(254,287)
#b=np.array((1,5,1000))
#b=np.array(( 1.40002561e-01,   8.51478121e+00,   2.62606729e+03))

# 3 model parameters below:

# 0th: how much power the turbine removes at the centre of the peak of the power distribution.
        # UNITS: [N/A]
# 1st: how wide the angle of effect is. UNITS: degrees
# 2nd: up to how far back the effect takes place (approx. 2.6km) UNITS: [m]
model=np.array((1.39998719e-01, 8.51483871e+00, 2.62613638e+03))

ws=2.0 #weibull scale factor
wei_gamma=gamma(1.+1./ws)

Pr = 2*10**6 #Rated Power for Horns Rev 1 Turbines. Max power output [Watts]
Vc = 4.0 #Cut-in Velocity. Starts producing energy at wind speed of 4m/s [m/s]
Vr = 15.0 #Rated Velocity. Starts producing max energy at 15m/s
Vf = 25.0 #Cut-off Velocity. Turbines cut out at wind speeds of 25m/s to prevent damage to the turbines.
k = 2.0 #Weibull shape parameter
(Pr,Vc,Vr,k)

#interpolation parameters
dvel=1.5 #[m/s]
dang=5. #[degrees]

In [ ]:
### (gives current position of the Horns Rev 1 wind turbines).

v=np.loadtxt('hornsrev_data_all.txt') # assigns data to variable v.

vxref=v[:,3] # (INDEXING STARTS FROM 0). Third column of data is velocity of wind in x direction.
vyref=v[:,4] # (INDEXING STARTS FROM 0). Fourth column of data is velocity of wind in y direction.
angles=v[:,0] # Zeroth column of data is wind angle.
vmean=np.sqrt(np.square(vxref)+np.square(vyref)) # Uses pythagoras to find the wind magnitude + direction for each location.

# print(vmean)
vmean=pd.DataFrame(vmean) # organises vmean into labelled data structure
# print(vmean)

vref=pd.DataFrame()

vmean['angle']=angles # add another column to vmean (angle)
vref=vref.append(vmean) # add empty pandas data frame

vref=vref.groupby('angle')
vref.groups
vref.describe()
vref.get_group(260).iat[50,0] # data manipulation to group by angle etc.
#vref.iat[5,0]
#vref.groupby('angle')

In [ ]:
### Imports, manipulates and displays coordinates of all turbines at Horns Rev 1.

# in coords matrix the first index references the turbine, the second index references said coordinate's value (0=x, 1=y).

coords_in = np.loadtxt('coords.txt')

# CLARIFY WHAT THESE COORDS ARE WITH PRINT: LOWER LEFT COORDS BELOW: and print a gap between outputs

print(min(coords_in[:,0]), min(coords_in[:,1])) # finds and displays most west and most south coordinates of turbines
# (note: not necessarily 1 turbine that represemts x and y values, just finds reference frame for wind farm). 

# sets xmin and ymin to these lower left most coordinate values.
xmin = min(coords_in[:,0])
ymin = min(coords_in[:,1])

# sets centre point halfway between the min and max coords:
xzero = (max(coords_in[:,0])-xmin)/2
yzero = (max(coords_in[:,1])-ymin)/2
# print(xzero, yzero)

# transforms coordinates so they're centred around the origin of the coordinates system:
coords_x=coords_in[:,0]-xzero-xmin
coords_y=coords_in[:,1]-yzero-ymin
coords=[coords_x,coords_y]



print (coords) # displays array of manipulated coordinates.


coords=np.array(coords).T # transposes coordinates
nturb=np.shape(coords)[0] # counts total number of turbines by  putting array into a tuple and counting entries.


plt.scatter(coords[:,0], coords[:,1]) # plots turbines as a scatter plot (x,y).



#Arrays for pairwise distances and angles
#Angle 0: x direction. Angle pi/2: y directıon

# calculates distance and angle between each pair of turbines:

distance=np.zeros((nturb,nturb)) # 2x2 matrix of distances between turbines i and j where distance i-i and distance j-j = 0
# as they're distances to themselves: i-i i-j
                                 #    j-i j-j

angle=np.zeros((nturb,nturb)) # same as above but for angles between turbines.
for i in range(0,nturb):
   
    # 80x80 matrices as there's 80 turbines at Horns Rev 1.
    
    # squares y distance and x distance then sqrt to find overall distance between 2 turbines.
    distance[i,:]=np.sqrt(np.square(coords[i,0]-coords[:,0])+np.square(coords[i,1]-coords[:,1]))
    # same as above using arctan2 whilst giving correct quadrant (between 2 turbines).
    angle[i,:]=np.arctan2(coords[:,1]-coords[i,1],coords[:,0]-coords[i,0])
# Rotate angles so that north=0 and convert to degrees (and clockwise instead of anticlockwise)
angle=-np.rad2deg(angle)+270
angle[0,1]

# Rotating angles to wind direction
windangle=5 # degrees
rotangles=np.mod(angle-windangle+180,360)-180

# creates list of turbines with largest to shortest distances between them.
(np.sort(distance))[0,2]

In [ ]:
# imports wind direction (meteorological data where north = 0 degrees)
import pandas as pd
data = pd.read_csv ('Directiondata.csv')
print (data)
# velocity is the mean velocity with which wind from this direction blows
# c determines how wide the distribution is
# frequency/100 determines how often the wind is blowing at this angle

In [ ]:
# could make comments here a markdown cell??

# interpolate 'velocity'
# info is only given every 30 degrees, giving a 'step function'
# interpolation gives higher resolution
# function is periodic as 360deg is the same as 0deg.
x = data['Angle']
y = data['Velocity']
from scipy.interpolate import CubicSpline as SP
import matplotlib.pyplot as plt
wbvel = SP (x, y,bc_type='periodic')
xs = np.arange(0, 360,0.10)
plt.plot(x,y, label='Line')
plt.plot(xs, wbvel(xs), label="Interpolated line ")
plt.title('Interpolated Velocity Data')
plt.xlabel('Angle')
plt.ylabel('Velocity')
plt.show()

# blue lines shows linear interpolation, orange shows smooth interpolation including across the
# periodic boundary.

In [ ]:
#interpolate 'c'
# info is only given every 30 degrees, giving a 'step function'
# interpolation gives higher resolution
# function is periodic as 360deg is the same as 0deg.
x = data['Angle']
y = data['c']
from scipy.interpolate import CubicSpline as SP
import matplotlib.pyplot as plt
wbshape = SP (x, y,bc_type='periodic')
xs = np.arange(0, 360, 0.1)
plt.plot(x,y)
plt.plot(xs, wbshape(xs), label="S")
plt.title('Interpolated Shape Parameter Data')
plt.xlabel('Angle')
plt.ylabel('Shape Parameter')
plt.show()

In [ ]:
#interpolate 'frequency'
# info is only given every 30 degrees, giving a 'step function'
# interpolation gives higher resolution
# function is periodic as 360deg is the same as 0deg.
x = data['Angle']
y = data['frequency']/(100.*30.) #Data is in percent per 30 degrees, changing to probability per degree
from scipy.interpolate import CubicSpline as SP
import matplotlib.pyplot as plt
windfreq = SP (x, y,bc_type='periodic')
xs = np.arange(0, 360, 0.1)
plt.plot(x,y)
plt.plot(xs, windfreq(xs), label="S")
plt.title('Interpolated Probability Data')
plt.xlabel('Angle')
plt.ylabel('Probability')
plt.show()
# plot shows wind direction is predominantly 200-320deg.

In [ ]:
# Simple Attenuation fn
# Determines how much wind there is AFTER a turbine.
def att(dist, ang, model): 
        # 'Model' parameters are defined in 2ND CELL.

    # angular part
    angular=np.where(2.*model[1]*np.abs(ang)<np.pi,np.square(np.cos(model[1]*ang)),0.)
    # angular = np.cos(model[1]*ang WHEN 2.*model[1]*np.abs(ang) is less than pi, else angular = 0.
    
    # radial part (distance) (Gaussian Function)
    radial=np.exp(-np.square(dist/model[2])) # decreasing exponential of square, scaled by 2nd parameter
    
    #penalty=np.exp(-np.square(dist/200))
    penalty = 0
    return 1.0-1*model[0]*angular*radial-2*model[0]*penalty # OUTCOME

In [ ]:

azimuths = np.radians(np.linspace(0, 360, 360)) # angles theta from 0 to 360 degress in 1 degree increments. Grid
zeniths = np.arange(0, 4000, 50) # distance from 0 to 4000 broken up into increments of 50.

r, theta = np.meshgrid(zeniths, azimuths)
values = att(r,np.mod(theta+np.pi,np.pi*2)-np.pi,model) # gets angles from -pi to pi, not from 0 to 2pi.

#print(values)
#-- Plot... ------------------------------------------------
fig, ax = plt.subplots(subplot_kw=dict(projection='polar'))
ax.contourf(theta, r, np.maximum(values,0.85*np.ones(np.shape(values))))

plt.show()
print(np.min(values))

## Markdown cell description:
# Polar plot is for 1 turbine, blue area shows where wind speed is affected. (Numbers label distance from turbine).
# Across the curve the distribution is the hump of cosine^2 function.
# Radially the distribution shown is Gaussian.
# wake is too narrow close to the turbine as it treats the turbine as a point rather than accommodating
# for the blades' diameter.

In [ ]:
# print(theta,np.mod(np.deg2rad(theta)+np.pi,np.pi*2)-np.pi)

In [ ]:
# r=np.arange

In [ ]:
# ind=np.zeros((8,10))
# for i in range(0,8):
#     for j in range(0,10):
#         ind[i,j]=i+8*j
# ind=ind.astype(int)
# ind

In [ ]:
# i=0
# j=4
# print('Turbine {} x {} y {}'.format(i,coords[i,0],coords[i,1]))
# print('Turbine {} x {} y {}'.format(j,coords[j,0],coords[j,1]))
# print('{}-{} dist {} angle {}'.format(i,j,distance[i,j], angle[i,j]))

In [ ]:
# isn't useful for continuing code, just rotates the windfarm.
def rotate(angle,coords):
    angle = np.pi*angle/180.
    rotcoordx = []
    rotcoordy = []
    for coord in coords:
        rotcoordx+=[coord[0]*np.cos(angle)-coord[1]*np.sin(angle)]
        rotcoordy+=[coord[0]*np.sin(angle)+coord[1]*np.cos(angle)]
    rotcoords=[rotcoordx,rotcoordy]
    rotcoords=np.array(rotcoords).T   
    return rotcoords 
#print rotate(254,coords)[:,0]
plt.scatter(rotate(0,coords)[:,0],rotate(0,coords)[:,1])

In [ ]:
# OPTIMISATION FUNCTION returning total deviation between model and training set.
#Total att
def deviation(b):
    total_att=np.ones((nturb,nwind))
    deviation=0
    for k in range(0,nwind):
        for j in range(0,nturb):
            for i in range(0,nturb):
                if (i!=j):
                    total_att[j,k] = total_att[j,k]*att(distance[i,j],np.mod(np.deg2rad(angle[i,j]-wind[k])+np.pi,np.pi*2)-np.pi,b)
 #           print('{0} {1} {2}'.format(j,wind[k],total_att[j,k]))
            deviation=deviation+np.square(vref.get_group(wind[k]).iat[j,0]-v0*total_att[j,k])
    return deviation

In [ ]:
# OPTIMISATION FUNCTION returning deviation for each turbine.
#Total att
def deviation_sq(model):
    total_att=np.ones((nturb,nwind))
    dev_vec=np.zeros((nturb*nwind))
    for k in range(0,nwind):
        for j in range(0,nturb):
            for i in range(0,nturb):
                if (i!=j):
                    total_att[j,k] = total_att[j,k]* att(distance[i,j],np.mod(np.deg2rad(angle[i,j]-wind[k])+np.pi,np.pi*2)-np.pi,model)
 #           print('{0} {1} {2}'.format(j,wind[k],total_att[j,k]))
            dev_vec[nturb*k+j]=(vref.get_group(wind[k]).iat[j,0]-v0*total_att[j,k])
    return dev_vec

In [ ]:
# USEFUL OUTPUT
# Calculates the % of wind that arrives at a turbine for a certain angle.
# Starts with assumption that there's zero wake so all turbines get full wind.
#talked about
#Total att
def windspeedreduction(distance,angle,direction,model): # vectors
    ndir=np.size(direction)
    total_att=np.ones((nturb,ndir)) # as many rows as there are turbines, as many columns as there are wind directions.
 #   power_vec=np.zeros((nturb,ndir))
    for j in range(0,nturb):
        for i in range(0,nturb):
            if (i!=j):
                # compounding wind speed reductions
                total_att[j,:] = total_att[j,:]* att(distance[i,j],np.mod(np.deg2rad(angle[i,j]-direction[:])+np.pi,np.pi*2)-np.pi,model)
  #      power_vec[j,:]=(v0*total_att[j,:])**3
    return total_att # matrix output that's needed. n turbine rows, n direction columns.
                     # Tells how much of 100% of wind arrives at a certain turbine from a certain direction.

In [ ]:
# Calculates the power produced by turbines when it sees a certain wind speed at a certain angle. 3D matrix
def power(wsr,v): # wind speed reduction, velocity
    nvel=np.size(v)
    nangle=np.size(wsr,1)
    power_vec=np.zeros((nturb,nangle,nvel))
    power_vec=POvec(np.outer(wsr,v)).reshape(nturb,nangle,nvel) # np.outer takes every wsr element and individually multiplies it with every element of v.
    return power_vec

In [ ]:
# gives probability of having wind speed v at a certain angle.
def windspeedprobability(angles,v,d_angle,d_vel):
    nvel=np.size(v) 
    nangle=np.size(angles)
    wsprob=np.zeros((nangle,nvel))
    for i in range(angles.shape[0]):
        # Get Weibull parameters for angle and evaluate probability
        wsprob[i,:]=(wei(v[:],wbvel(angles[i]),wbshape(angles[i])))*windfreq(angles[i])*d_angle*d_vel
    return wsprob

In [ ]:
# Defines Weibull distribution (which shows the probability of any given wind speed occuring).

def wei(x,n,k): # convention to have in this order, scale parameter comes first (x) 
    u=n/gamma(1+1/k) #scaled wind speed
    return (k / u) * (x / u)**(k - 1) * np.exp(-(x / u)**k)
# k = Weibull shape parameter
# n = scale parameter
# x= value we are valuating

In [ ]:
def wind_dist(v,vm):
    return wei(v,vm,k)

In [ ]:
wind_dist(1.0,v0)

In [ ]:
# Plots Weibull distribution.

x=np.arange(0,25,0.1)
y=wind_dist(x,v0)

plt.xlabel('Wind Velocity')
plt.ylabel('Probability')
plt.plot(x,y)

In [ ]:
velocities=np.arange(Vc,Vf,dvel) # 1D array ranging from Vc to Vf in dvel intervals.
angles=np.arange(0,360,dang) # 0 to 360 with intervals of dang.
wsp=windspeedprobability(angles,velocities,dang,dvel) # tells probability of getting wind from a certain direction with a certain velocity.
# sum of all numbers would add up to 1.
print(wsp,np.shape(wsp),np.sum(wsp))
#plt.plot(x,y)

In [ ]:
# Calculates the power curve of a turbine.

# Plots the power curve of a turbine.
# No power output from 0-4m/s, then cubic rise up to 15m/s and stays
# constant until cut-off velocity of 25m/s.

#cubic Based Power output
a=Pr/(Vr**3-Vc**3)
b=Vc**3/(Vr**3-Vc**3)

def q(v): # q is power output

    if (v<Vc): # below cut in velocity power is 0.
        q=0
    elif (v<Vr):
        q=a*v**3-b*Pr # cubic power output between cut in and max.
    elif (v<Vf):
        q=Pr # max power between max and cut off velocity.
    else: 
        q=0 # no power above cut off velocity.
    return q

In [ ]:
x=np.arange(0,25,0.1)
POvec=np.vectorize(q) # vectorises power output function q (from cell above).
y=POvec(x)

plt.xlabel('Wind Velocity')
plt.ylabel('Power')
plt.plot(x,y)

In [ ]:
wsr=windspeedreduction(distance,angle,angles,model)
powout=power(wsr,velocities)
#print(powout,np.shape(powout))

In [ ]:
# Expected turbine output (Watts) in Horns Rev 1 for the given wind distributions
# and positions.Can be seen that top left (NW) produces most energy.

# EXPECTED POWER FROM WIND FARM CALCULATIONS
# multiply the power that you'd get for each wind angle and speed condition by the probability of this occuring, for all angles and speed
# values and sum to find the expected power.
# Sum up this value for all turbines to find total wind farm expected power output.
output=np.zeros((nturb)) 
output=np.tensordot(powout,wsp,2)
print(output,np.shape(output),np.sum(output))

# last output is total power output in Watts.

In [ ]:
# What all turbines would output (Watts) with no interference.
#Reference output:
# Same calculations as cell above except each turbine sees the full amount of wind every time
# (i.e. no wind reduction from other turbines).
refpowout=power(np.ones((nturb,np.size(angles))),velocities)
refoutput=np.zeros((nturb))
refoutput=np.tensordot(refpowout,wsp,axes=2)
print(refoutput,np.shape(refoutput),np.sum(refoutput))

# last output is total power output in Watts.

In [ ]:
# assigns smaller interpolation intervals to dvel and dang to compare to coarser interpolations.
dvel=0.1
dang=0.1
velocities=np.arange(Vc,Vf+dvel,dvel)
angles=np.arange(0,360,dang)
wsp=windspeedprobability(angles,velocities,dang,dvel)
wsr=windspeedreduction(distance,angle,angles,model)
powout=power(wsr,velocities)
output=np.zeros((nturb)) 
output=np.tensordot(powout,wsp,2)
print (dvel,dang,np.sum(output))
fineout=output

In [ ]:
# Calculates RMS error from changing interpolation value.
velarray=np.array([1.5])
angarray=np.array([5])
rmserror=np.zeros((np.size(velarray),np.size(angarray)))
i=0
for dvel in velarray:
    j=0
    for dang in angarray:
        velocities=np.arange(Vc,Vf+dvel,dvel)
        angles=np.arange(0,360,dang)
        wsp=windspeedprobability(angles,velocities,dang,dvel)
        wsr=windspeedreduction(distance,angle,angles,model)
        powout=power(wsr,velocities)
        output=np.zeros((nturb)) 
        output=np.tensordot(powout,wsp,2)
        rmserror[i,j]=np.sqrt(np.sum(np.square(output-fineout))/np.size(output))
        print (dvel,dang,np.sum(output),rmserror[i,j])
        j+=1
    i+=1
print(rmserror)

In [ ]:
# creates input needed for MST, similar recreation of cell 4. (Could make cell 4 a function then call it here).
def geom_analysis(coords):
    #Arrays for pairwise distances and angles
    #Angle 0: x directıon. Angle pi/2: y directıon
    nturb=np.size(coords,0)
    distance=np.zeros((nturb,nturb))
    angle=np.zeros((nturb,nturb))
    for i in range(0,nturb):
        distance[i,:]=np.sqrt(np.square(coords[i,0]-coords[:,0])+np.square(coords[i,1]-coords[:,1]))
        angle[i,:]=np.arctan2(coords[:,1]-coords[i,1],coords[:,0]-coords[i,0])
        distance[i,i]=1e10
#Rotate angles so that north=0 and convert to degrees
    angle=-np.rad2deg(angle)+270
    return distance,angle

In [ ]:
# determines minimum amount of cable needed to connect all turbines.
# creates lower bound, ignoring redundancy and load balancing, etc.
# if 1 connection is cut, you bisect tree in 2 different parts, could be improved.
def Minimum_Spanning_Tree(distance):
#    edges= np.zeros((0,3))
    g=Graph(nturb)
    for i in range(0,nturb):
        for j in range(i+1,nturb):
#            if distance[i,j]<1500:
            g.addEdge(i,j,int(distance[i,j]*1000))
#    for p in range(0,(edges.shape[1]-1)):
#        if int(edges[p,2])<1500:
#            g.addEdge(int(edges[p,0]),int(edges[p,1]),int(edges[p,2]))
    MSTweight=g.boruvkaMST()
#    print (MSTweight)
    return MSTweight

In [ ]:
# analyses the coordinates tocollate the distances and angles between all of them
# calculates the minimum spanning tree length and adds the assumption of cabling costs,
# turbines costs, maintenance costs and fixed costs.

def targetfunction(coord_flat):
#from scipy.optimize import minimize
    coords=np.reshape(coord_flat,(-1,2)) # undoes the coordinate flattening
    nturb=np.size(coords,0)
#    print(coords)
    distance,angle = geom_analysis(coords)
    MSTweight=Minimum_Spanning_Tree(distance)###
    CableCost=(MSTweight[0]/1000)*300###
    
    ### REMOVE THIS LINE BELOW. ISN't NEEDED!!!
    
    coords_in = np.loadtxt('coords.txt')###
    
    ##########################################
    
    # technically could add these at end as they add slight computation itme but this is neglibile. It's best to keep these
    # in as number of turbines can change. Especially if you then want to optimise the number of turbines.
    TurbineCost=(nturb)*3000000/1.2821###
    MaintenanceCosts=(nturb)*2*1563720###
    FixedCost=600000###
    # (coords) sets the area in which turbines can be placed.
    xmin = min(coords[:,0])###
    ymin = min(coords[:,1])###
    xmax = max(coords[:,0])###
    ymax = max(coords[:,1])###
    
    
    
    LandCost=(xmax-xmin)*(ymax-ymin)*(17245/10000)###
    TotalCost=CableCost+TurbineCost+LandCost+MaintenanceCosts+FixedCost###
    
    # remove penalty also? Redundant
    penalty=0
    
    wsr=windspeedreduction(distance,angle,angles,model)
    powout=power(wsr,velocities) # total output power
    output=np.tensordot(powout,wsp,2)
    
    print(np.min(distance),np.sum(output),TotalCost,(TotalCost/(np.sum(output))))###
# outputs cost per power e.g. £/watt.

# remove penalty also? Redundant

    #if(np.min(distance)<200): penalty=10000*nturb*(np.min(distance)-200)**2
    return (TotalCost/(np.sum(output)))#+penalty

In [ ]:
flat_coords=coords.flatten() # turns 2D coords into single list of data without any more structure.
bounds=Bounds(np.tile([-2728.5, -1952.],nturb), np.tile([2728.5, 1952.],nturb))
dvel=1.5
dang=5
velocities=np.arange(Vc,Vf+dvel,dvel)
angles=np.arange(0,360,dang)
#MSTweight=Minimum_Spanning_Tree(distance)
res = minimize(targetfunction, flat_coords, method='L-BFGS-B', jac=False, hess=None, options={'disp': 2, 'eps': 50}, bounds=bounds)

In [ ]:
#print(res.x)
#print(res.x-flat_coords)

In [ ]:
# Unflattens minimization function output (res.x) and puts them into newcoords
newcoords=np.reshape(res.x,(-1,2))
#print(newcoords)

In [ ]:
# plot to compare old and new coordinates

plt.scatter(newcoords[:,0],newcoords[:,1], color='red') # plots new coordinates
plt.scatter(rotate(0,coords)[:,0],rotate(0,coords)[:,1]) # coords are the original coordinates

plt.legend(["Optimised", "Original"],bbox_to_anchor=(0., 1.02, 1., .102), loc=3,
           ncol=2, mode="expand", borderaxespad=0.)

In [ ]:
newcoords_unconstrained=newcoords

In [ ]:
plt.scatter(rotate(0,coords)[:,0],rotate(0,coords)[:,1])

In [ ]:
plt.scatter(newcoords[:,0],newcoords[:,1], color='red')

In [ ]:
54123077.32294567-53477710.87598161

In [ ]:
((55751104.56647739-54166276.11371625)/55751104.56647739)*100

In [ ]:
# Could remove this as done twice
MSTweight=Minimum_Spanning_Tree(geom_analysis(newcoords)[0])

In [ ]:
# prints the connections
MSTweight=Minimum_Spanning_Tree(geom_analysis(newcoords)[0])
print (MSTweight)
links=MSTweight[1]
print(links)

In [ ]:
# MST plot of NEW COORDS (analysis is above)
plt.scatter(newcoords[:,0],newcoords[:,1])
plt.axis('equal')
for i in links:
    plt.plot(newcoords[i[:],0],newcoords[i[:],1])

In [ ]:
print(MSTweight[0])

In [ ]:
coords_in = np.loadtxt('coords.txt')###
print(int(np.shape(coords_in[:,0])[0]))

In [ ]:
# MST analysis and plot of OLD COORDS

MSTweight=Minimum_Spanning_Tree(geom_analysis(coords)[0])
links=MSTweight[1]
plt.scatter(coords[:,0],coords[:,1])
for i in links:
    plt.plot(coords[i[:],0],coords[i[:],1])#ignore this one

In [ ]:
# displays the last contents of the target function, could tidy this up by calling target function!

coords=np.loadtxt('coords.txt')
nturb=np.size(coords,0)
distance,angle = geom_analysis(coords)
MSTweight=Minimum_Spanning_Tree(distance)###
CableCost=(MSTweight[0]/1000)*300###
TurbineCost=(nturb)*3000000/1.2821###
MaintenanceCosts=(nturb)*2*1563720###
FixedCost=600000###
xmin = min(coords[:,0])###
ymin = min(coords[:,1])###
xmax = max(coords[:,0])###
ymax = max(coords[:,1])###

    
LandCost=(xmax-xmin)*(ymax-ymin)*(17245/10000)###


TotalCost=CableCost+TurbineCost+LandCost+MaintenanceCosts+FixedCost###
penalty=0
wsr=windspeedreduction(distance,angle,angles,model)
powout=power(wsr,velocities)
output=np.tensordot(powout,wsp,2)

print(np.min(distance),np.sum(output),TotalCost,(TotalCost/(np.sum(output))))###

In [ ]:
#topodata = open("nsea86.m77t")
#print(topodata.read()) 
#type(topodata)
data = pd.read_csv("nsea86.m77t", sep='\t')
print(data)

In [ ]:
#drop irrelevant columns in this cell

In [ ]:
# TESTING

coordinate = utm.from_latlon(54, 7) # inputs are lat, long
print (coordinate)
coordinate2 = utm.from_latlon(54.1, 7)
print (coordinate2) # outputs are easting, northing
print(coordinate[0] - coordinate2[0])
print(coordinate[1] - coordinate2[1])

In [ ]:
#data['coordinate'] = utm.from_latlon(data['LAT'], data['LON'])

data['easting'] = data.apply(lambda row: utm.from_latlon(row['LAT'], row['LON'])[0], axis=1)
data['northing'] = data.apply(lambda row: utm.from_latlon(row['LAT'], row['LON'])[1], axis=1)

#coordinates are approximate, explain further
#max - min/2 to find centre and map to 0,0.
#bivariate spline

print(data)


In [ ]:
depth = spline.SmoothBivariateSpline(data['easting'], data['northing'], data['CORR_DEPTH'])
#attempt plotting to see if it is smoothing too much

#(x, y, z, w=None, bbox=[None, None, None, None], kx=3, ky=3, s=None, eps=1e-16)

In [ ]:
print(depth(592318.3199199219, 6040378.688430359))
print(depth.get_coeffs)


# create 2 arrays from min to max of eastings and northings and create x y mesh grid
# eastings and northings instead of azimuths and zeniths

ax = plt.axes(projection='3d')
#ax.plot_trisurf(X, Y, Z, linewidth=0, antialiased=False)
ax.plot_trisurf(data['easting'], data['northing'], data['CORR_DEPTH'], cmap='viridis', edgecolor='none')
ax.set_title('surface');

# depth not mod, contour function not polar plot
# optional argument s in smoothbivariatespline can be looked into as we may be oversmoothing.
# Must do some smoothing still, may need a different interpolation scheme. SciPi interpolate has many
# multivariate interpolation schemes. Can work with it for now and replace it at a later stage, can
# go back and replace later.


#azimuths = np.radians(np.linspace(0, 360, 360)) # angles theta from 0 to 360 degress in 1 degree increments. Grid
#zeniths = np.arange(0, 4000, 50) # distance from 0 to 4000 broken up into increments of 50.

#r, theta = np.meshgrid(zeniths, azimuths)
#values = att(r,np.mod(theta+np.pi,np.pi*2)-np.pi,model) # gets angles from -pi to pi, not from 0 to 2pi.


#fig, ax = plt.subplots(subplot_kw=dict(projection='polar'))
#ax.contourf(theta, r, np.maximum(values,0.85*np.ones(np.shape(values))))

#plt.show()
#print(np.min(values))